# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [106]:
# download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet'])

# import libraries
import re
import pickle
import pandas as pd
import numpy  as np
from nltk.tokenize                   import word_tokenize, WhitespaceTokenizer
from nltk.stem                       import WordNetLemmatizer
from scipy.stats.mstats              import gmean
from sqlalchemy                      import create_engine
from sklearn.base                    import BaseEstimator, TransformerMixin
from sklearn.pipeline                import Pipeline, FeatureUnion
from sklearn.ensemble                import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics                 import make_scorer, accuracy_score, f1_score, fbeta_score, classification_report
from sklearn.multioutput             import MultiOutputClassifier
from sklearn.model_selection         import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haiqi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\haiqi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [107]:
# load data from database
engine = create_engine('sqlite:///disaster_response_ETL.db')
df = pd.read_sql_table('message_categories', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [109]:
# Run this code cell to check how many null values are in the data set
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [110]:
X = df['message']
y = df.iloc[:,4:]
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [114]:
def tokenize(text):
    tokens     = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [105]:
pipeline = Pipeline([('vect',  CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',   MultiOutputClassifier(RandomForestClassifier()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [115]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [116]:
#Train pipeline
pipeline.fit(X_train, y_train)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: F

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [133]:
def multioutput_fscore(y_true,y_pred,beta=1):
    score_list = []
    if isinstance(y_pred, pd.DataFrame) == True:
        y_pred = y_pred.values
    if isinstance(y_true, pd.DataFrame) == True:
        y_true = y_true.values
    for column in range(0,y_true.shape[1]):
        score = fbeta_score(y_true[:,column],y_pred[:,column],beta,average='weighted')
        score_list.append(score)
    f1score_numpy = np.asarray(score_list)
    f1score_numpy = f1score_numpy[f1score_numpy<1]
    f1score = gmean(f1score_numpy)
    return  f1score

def scoring_model(y_true, y_pred):
    multioutput_f1   = multioutput_fscore(y_true, y_pred, beta = 1)
    overall_accuracy = (y_pred == y_true).mean().mean()

    print('Average overall accuracy:     {0:.2f}%\n'.format(overall_accuracy*100))
    print('F1 score (custom definition): {0:.2f}%\n'.format(multioutput_f1*100))
    
    for i in range(len(category_names)):
        print('------------------------------------------------------\n')
        print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i], y_pred[:, i]))


scorer = make_scorer(multioutput_fscore,greater_is_better = True)

In [131]:
y_pred = pipeline.predict(X_test)

In [132]:
scoring_model(y_test, y_pred)

Average overall accuracy:     94.22%

F1 score (custom definition): 92.34%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.60      0.34      0.43      1533
           1       0.81      0.93      0.87      4973
           2       0.38      0.19      0.25        48

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.60      0.48      0.52      6554
weighted avg       0.76      0.79      0.76      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.94      5440
           1       0.84      0.42      0.56      1114

   micro avg       0.89      0.89      0.89      6554
   macro avg       0.86      0.70      0.75      6554
weighted avg       0.88      0.89      0.87      6554

------------------------------------------------------

Category

### 6. Improve your model
Use grid search to find better parameters. 

In [134]:
parameters = {  'vect__ngram_range':  [(1,1),(1,2)],
                'vect__max_df':       (0.5, 0.75, 1.0),
                'vect__max_features': (None, 5000, 10000),
                'tfidf__use_idf':     (True, False),
             }

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer,verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 13.8min finished
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 1), (1, 2)], 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000, 10000), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(multioutput_fscore), verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [135]:
y_pred = cv.predict(X_test)

In [136]:
scoring_model(y_test, y_pred)

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Average overall accuracy:     94.71%

F1 score (custom definition): 93.28%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.58      0.36      0.45      1533
           1       0.82      0.92      0.87      4973
           2       0.36      0.33      0.35        48

   micro avg       0.78      0.78      0.78      6554
   macro avg       0.59      0.54      0.55      6554
weighted avg       0.76      0.78      0.76      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94      5440
           1       0.82      0.51      0.63      1114

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------

Category

C:\Users\haiqi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [138]:
# StartingVerbExtractor Class Definition
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [139]:
pipeline = Pipeline([('features', FeatureUnion([('text_pipeline', Pipeline([
                                                                            ('vect',  CountVectorizer(tokenizer=tokenize)),
                                                                            ('tfidf', TfidfTransformer()),
                                                                          ])),
                                                ('starting_verb', StartingVerbExtractor())
                                              ])
                     ),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier()))
                   ])

In [140]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

In [141]:
y_pred = pipeline.predict(X_test)

In [142]:
scoring_model(y_test, y_pred)

Average overall accuracy:     94.75%

F1 score (custom definition): 93.62%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.53      0.21      0.30      1533
           1       0.79      0.94      0.86      4973
           2       0.27      0.12      0.17        48

   micro avg       0.76      0.76      0.76      6554
   macro avg       0.53      0.42      0.44      6554
weighted avg       0.73      0.76      0.72      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      5440
           1       0.78      0.56      0.65      1114

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.85      0.77      0.80      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------

Category

Category: transport 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6251
           1       0.69      0.21      0.32       303

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.83      0.60      0.65      6554
weighted avg       0.95      0.96      0.95      6554

------------------------------------------------------

Category: buildings 
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      6244
           1       0.66      0.39      0.49       310

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.81      0.69      0.74      6554
weighted avg       0.96      0.96      0.96      6554

------------------------------------------------------

Category: electricity 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6418
           1       0.57      0.26      0.36       136

  

In [147]:
parameters = {
        'features__text_pipeline__vect__ngram_range':  ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df':       (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf':     (True, False),
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer,verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 20.8min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed: 75.2min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, ma...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0), 'features__text_pipeline__vect__max_features': (None, 5000, 10000), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(multioutput_fscore), verbose=2)

In [148]:
y_pred = cv.predict(X_test)

In [149]:
scoring_model(y_test, y_pred)

Average overall accuracy:     94.79%

F1 score (custom definition): 93.69%

------------------------------------------------------

Category: related 
               precision    recall  f1-score   support

           0       0.52      0.22      0.31      1533
           1       0.79      0.93      0.86      4973
           2       0.18      0.10      0.13        48

   micro avg       0.76      0.76      0.76      6554
   macro avg       0.50      0.42      0.43      6554
weighted avg       0.72      0.76      0.72      6554

------------------------------------------------------

Category: request 
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      5440
           1       0.79      0.57      0.66      1114

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.85      0.77      0.80      6554
weighted avg       0.89      0.90      0.89      6554

------------------------------------------------------

Category

Category: transport 
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6251
           1       0.79      0.20      0.32       303

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.88      0.60      0.65      6554
weighted avg       0.95      0.96      0.95      6554

------------------------------------------------------

Category: buildings 
               precision    recall  f1-score   support

           0       0.97      0.99      0.98      6244
           1       0.66      0.41      0.51       310

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.82      0.70      0.74      6554
weighted avg       0.96      0.96      0.96      6554

------------------------------------------------------

Category: electricity 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6418
           1       0.61      0.30      0.40       136

  

### 9. Export your model as a pickle file

In [151]:
pickle.dump(cv, open('model.p', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.